# Demo for MoleculeSTM pretraining

All the scripts can be found in `MoleculeSTM/pretrain.py`.

## 1 Load and Customize Arguments

In [2]:
# Set-up the environment variable to ignore warnings
import warnings
warnings.filterwarnings('ignore')
import os
os.environ['TOKENIZERS_PARALLELISM'] = 'False'

import argparse

parser = argparse.ArgumentParser()

parser.add_argument("--seed", type=int, default=42)
parser.add_argument("--device", type=int, default=0)

parser.add_argument("--dataspace_path", type=str, default="../data")
parser.add_argument("--dataset", type=str, default="PubChemSTM1K")
parser.add_argument("--text_type", type=str, default="SciBERT", choices=["SciBERT"])
parser.add_argument("--molecule_type", type=str, default="SMILES", choices=["SMILES", "Graph"])

parser.add_argument("--batch_size", type=int, default=4)
parser.add_argument("--text_lr", type=float, default=1e-4)
parser.add_argument("--mol_lr", type=float, default=1e-4)
parser.add_argument("--text_lr_scale", type=float, default=0.1)
parser.add_argument("--mol_lr_scale", type=float, default=0.1)
parser.add_argument("--num_workers", type=int, default=8)
parser.add_argument("--epochs", type=int, default=100)
parser.add_argument("--decay", type=float, default=0)
parser.add_argument("--verbose", type=int, default=1)
parser.add_argument("--output_model_dir", type=str, default=None)

########## for SciBERT ##########
parser.add_argument("--max_seq_len", type=int, default=512)

########## for MegaMolBART ##########
parser.add_argument("--megamolbart_input_dir", type=str, default="../data/pretrained_MegaMolBART/checkpoints")

########## for contrastive SSL ##########
parser.add_argument("--SSL_loss", type=str, default="EBM_NCE", choices=["EBM_NCE", "InfoNCE"])
parser.add_argument("--SSL_emb_dim", type=int, default=256)
parser.add_argument("--CL_neg_samples", type=int, default=1)
parser.add_argument("--T", type=float, default=0.1)
parser.add_argument('--normalize', dest='normalize', action='store_true')
parser.add_argument('--no_normalize', dest='normalize', action='store_false')
parser.set_defaults(normalize=True)

args = parser.parse_args("")
print("arguments\t", args)

arguments	 Namespace(CL_neg_samples=1, SSL_emb_dim=256, SSL_loss='EBM_NCE', T=0.1, batch_size=4, dataset='PubChemSTM1K', dataspace_path='../data', decay=0, device=0, epochs=100, max_seq_len=512, megamolbart_input_dir='../data/pretrained_MegaMolBART/checkpoints', mol_lr=0.0001, mol_lr_scale=0.1, molecule_type='SMILES', normalize=True, num_workers=8, output_model_dir=None, seed=42, text_lr=0.0001, text_lr_scale=0.1, text_type='SciBERT', verbose=1)


## 2 Load Packages

In [3]:
import os
import time
import numpy as np
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader as torch_DataLoader

from torch_geometric.loader import DataLoader as pyg_DataLoader
from transformers import AutoModel, AutoTokenizer

from MoleculeSTM.datasets import (
    PubChemSTM_Datasets_SMILES, PubChemSTM_SubDatasets_SMILES,
    PubChemSTM_Datasets_Graph, PubChemSTM_SubDatasets_Graph,
    PubChemSTM_Datasets_Raw_SMILES, PubChemSTM_SubDatasets_Raw_SMILES,
    PubChemSTM_Datasets_Raw_Graph, PubChemSTM_SubDatasets_Raw_Graph
)
from MoleculeSTM.models import GNN, GNN_graphpred
from MoleculeSTM.utils import prepare_text_tokens, get_molecule_repr_MoleculeSTM, freeze_network
from MoleculeSTM.models.mega_molbart.mega_mol_bart import MegaMolBART

[2024-04-02 20:18:58,964] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)


NOTE: Redirects are currently not supported in Windows or MacOs.


ModuleNotFoundError: No module named 'apex.normalization'

## 3 Supporting Functions

In [4]:
def cycle_index(num, shift):
    arr = torch.arange(num) + shift
    arr[-shift:] = torch.arange(shift)
    return arr


def do_CL(X, Y, args):
    if args.normalize:
        X = F.normalize(X, dim=-1)
        Y = F.normalize(Y, dim=-1)

    if args.SSL_loss == 'EBM_NCE':
        criterion = nn.BCEWithLogitsLoss()
        neg_Y = torch.cat([Y[cycle_index(len(Y), i + 1)] for i in range(args.CL_neg_samples)], dim=0)
        neg_X = X.repeat((args.CL_neg_samples, 1))

        pred_pos = torch.sum(X * Y, dim=1) / args.T
        pred_neg = torch.sum(neg_X * neg_Y, dim=1) / args.T

        loss_pos = criterion(pred_pos, torch.ones(len(pred_pos)).to(pred_pos.device))
        loss_neg = criterion(pred_neg, torch.zeros(len(pred_neg)).to(pred_neg.device))
        CL_loss = (loss_pos + args.CL_neg_samples * loss_neg) / (1 + args.CL_neg_samples)

        CL_acc = (torch.sum(pred_pos > 0).float() + torch.sum(pred_neg < 0).float()) / \
                 (len(pred_pos) + len(pred_neg))
        CL_acc = CL_acc.detach().cpu().item()

    elif args.SSL_loss == 'InfoNCE':
        criterion = nn.CrossEntropyLoss()
        B = X.size()[0]
        logits = torch.mm(X, Y.transpose(1, 0))  # B*B
        logits = torch.div(logits, args.T)
        labels = torch.arange(B).long().to(logits.device)  # B*1

        CL_loss = criterion(logits, labels)
        pred = logits.argmax(dim=1, keepdim=False)
        CL_acc = pred.eq(labels).sum().detach().cpu().item() * 1. / B

    else:
        raise Exception

    return CL_loss, CL_acc

## 4 Training Function

In [6]:
def train(
    epoch,
    dataloader,
    text_model, text_tokenizer,
    molecule_model, MegaMolBART_wrapper=None):

    text_model.train()
    molecule_model.train()
    text2latent.train()
    mol2latent.train()

    if args.verbose:
        L = tqdm(dataloader)
    else:
        L = dataloader
    
    start_time = time.time()
    accum_loss, accum_acc = 0, 0
    for step, batch in enumerate(L):
        description = batch[0]
        molecule_data = batch[1]

        description_tokens_ids, description_masks = prepare_text_tokens(
            device=device, description=description, tokenizer=text_tokenizer, max_seq_len=args.max_seq_len)
        description_output = text_model(input_ids=description_tokens_ids, attention_mask=description_masks)
        description_repr = description_output["pooler_output"]
        description_repr = text2latent(description_repr)

        molecule_data = list(molecule_data) # for SMILES_list
        molecule_repr = get_molecule_repr_MoleculeSTM(
            molecule_data, mol2latent=mol2latent,
            molecule_type=molecule_type, MegaMolBART_wrapper=MegaMolBART_wrapper)

        loss_01, acc_01 = do_CL(description_repr, molecule_repr, args)
        loss_02, acc_02 = do_CL(molecule_repr, description_repr, args)
        loss = (loss_01 + loss_02) / 2
        acc = (acc_01 + acc_02) / 2
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        accum_loss += loss.item()
        accum_acc += acc
    
    accum_loss /= len(L)
    accum_acc /= len(L)
    
    global optimal_loss
    temp_loss = accum_loss
    if temp_loss < optimal_loss:
        optimal_loss = temp_loss        
    print("CL Loss: {:.5f}\tCL Acc: {:.5f}\tTime: {:.5f}".format(accum_loss, accum_acc, time.time() - start_time))
    return

## 5 Start Pretraining

### 5.1 Set seed

In [7]:
torch.manual_seed(args.seed)
np.random.seed(args.seed)
device = torch.device("cuda:" + str(args.device)) \
    if torch.cuda.is_available() else torch.device("cpu")
# if torch.cuda.is_available():
torch.cuda.manual_seed_all(args.seed)

### 5.2 Prepare Text Model 

In [8]:
kwargs = {}

if args.text_type == "SciBERT":
    pretrained_SciBERT_folder = os.path.join(args.dataspace_path, 'pretrained_SciBERT')
    print("Download SciBert to {}".format(pretrained_SciBERT_folder))
    text_tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased', cache_dir=pretrained_SciBERT_folder)
    text_model = AutoModel.from_pretrained('allenai/scibert_scivocab_uncased', cache_dir=pretrained_SciBERT_folder).to(device)
    kwargs["text_tokenizer"] = text_tokenizer
    kwargs["text_model"] = text_model
    text_dim = 768
else:
    raise Exception

Download SciBert to ../data/pretrained_SciBERT


### 5.3 Start training MoleculeSTM-SMILES

#### 5.3.1 Prepare MegaMolBART (SMILES Model) and Dataset

In [9]:
dataset_root = os.path.join(args.dataspace_path, "PubChemSTM_data")
    
molecule_type = "SMILES"

dataset = PubChemSTM_SubDatasets_SMILES(dataset_root, size=1000)
dataloader_class = torch_DataLoader

if args.output_model_dir is not None:
    MegaMolBART_dir = os.path.join(args.output_model_dir, "SMILES")
else:
    MegaMolBART_dir = None
MegaMolBART_wrapper = MegaMolBART(
    vocab_path="../MoleculeSTM/bart_vocab.txt",
    input_dir=args.megamolbart_input_dir,
    output_dir=MegaMolBART_dir)
molecule_model = MegaMolBART_wrapper.model
kwargs["MegaMolBART_wrapper"] = MegaMolBART_wrapper
kwargs["molecule_model"] = molecule_model
molecule_dim = 256

dataloader = dataloader_class(dataset, batch_size=args.batch_size, shuffle=True, num_workers=args.num_workers)

FileNotFoundError: [Errno 2] No such file or directory: '../data/PubChemSTM_data/raw/CID2text.json'

#### 5.3.2 Prepare Two Projection Layers

In [10]:
text2latent = nn.Linear(text_dim, args.SSL_emb_dim).to(device)
mol2latent = nn.Linear(molecule_dim, args.SSL_emb_dim).to(device)

NameError: name 'molecule_dim' is not defined

#### 5.3.3 Prepare Optimizers

In [11]:
model_param_group = [
    {"params": text_model.parameters(), "lr": args.text_lr},
    {"params": molecule_model.parameters(), "lr": args.mol_lr},
    {"params": text2latent.parameters(), "lr": args.text_lr * args.text_lr_scale},
    {"params": mol2latent.parameters(), "lr": args.mol_lr * args.mol_lr_scale},
]
optimizer = optim.Adam(model_param_group, weight_decay=args.decay)
optimal_loss = 1e10

NameError: name 'molecule_model' is not defined

#### 5.3.4 Start Training

In [10]:
for e in range(3):
    print("Epoch {}".format(e))
    train(e, dataloader, **kwargs)

Epoch 0


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 250/250 [00:49<00:00,  5.02it/s]


CL Loss: 0.69800	CL Acc: 0.50400	Time: 49.79203
Epoch 1


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 250/250 [00:48<00:00,  5.14it/s]


CL Loss: 0.69504	CL Acc: 0.50450	Time: 48.61034
Epoch 2


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 250/250 [00:48<00:00,  5.13it/s]

CL Loss: 0.69426	CL Acc: 0.50175	Time: 48.72926
